In [12]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt

In [13]:
import glob

In [14]:
import numpy as np
import os
import glob
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

In [15]:
from csl_generator import getsigmas

In [16]:
from openpyxl import Workbook
from openpyxl.chart import Reference, LineChart
from openpyxl.chart.layout import Layout, ManualLayout
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.drawing.line import LineProperties

In [40]:
def make_excel(wb,ws,name,test_data):
    titles = [["angle","my_model","cnid_sampling","gbindice"]]
    test_data = titles + test_data

    for i,v in enumerate(test_data):
        ws.append(v)

    rmin = ws.min_row
    rmax = ws.max_row
    cmin = ws.min_column
    cmax = ws.max_column-1


    chart = LineChart()
    src = Reference(ws, min_col=cmin+1, min_row=rmin, max_col=cmax, max_row=rmax)
    chart.add_data(src, titles_from_data=True)
    cat = Reference(ws, min_col=cmin, min_row=rmin+1, max_row=rmax)  # 項目名の設定
    chart.set_categories(cat)
    chart.title = f"({name})"  # グラフタイトル
    chart.x_axis.title = 'x'  # 軸ラベル
    chart.y_axis.title = 'y'
    chart.anchor = 'A8'  # グラフの表示位置
    chart.width = 13  # グラフのサイズ
    chart.height = 8


    chart.layout = Layout(ManualLayout(x = 0.05, y = 0.05, h = 0.9, w = 0.9, xMode="edge", yMode="edge"))
    #プロットエリアの
    chart.plot_area.spPr = GraphicalProperties(solidFill = "f2ffff", ln =LineProperties(solidFill="000000", w=2*12700))
    chart.plot_area.spPr = GraphicalProperties(noFill = True)

    ws.add_chart(chart)
    wb.save('gbenergy.xlsx')

    # 線の色を変更する
    chart.ser[0].graphicalProperties.line.solidFill = "FF0000"
    chart.ser[1].graphicalProperties.line.solidFill = "00FF00"
    # chart.ser[2].graphicalProperties.line.solidFill = "0000FF"
    wb.save('gbenergy.xlsx')

    # 線種を変更する
    # 設定可能な値：'lgDash', 'sysDash', 'dashDot', 'solid', 'sysDashDot',
    # 'lgDashDotDot', 'dot', 'sysDot', 'sysDashDotDot', 'dash', 'lgDashDot'
    chart.ser[0].graphicalProperties.line.dashStyle = 'solid'
    chart.ser[1].graphicalProperties.line.dashStyle = 'solid'
    # chart.ser[2].graphicalProperties.line.dashStyle = 'solid'
    wb.save('gbenergy.xlsx')

    # 線の太さを変更する
    chart.ser[0].graphicalProperties.line.width = 10000
    chart.ser[1].graphicalProperties.line.width = 10000
    # chart.ser[2].graphicalProperties.line.width = 30000
    wb.save('gbenergy.xlsx')

    # 線を滑らかにする
    chart.ser[0].smooth = False
    wb.save('gbenergy.xlsx')

    # マーカーを変更する
    # 設定可能な値：'x', 'auto', 'picture', 'star', 'diamond', 'plus', 'dot',
    # 'square', 'dash', 'triangle', 'circle'
    chart.ser[0].marker.symbol = 'circle'
    chart.ser[1].marker.symbol = 'diamond'
    # chart.ser[2].marker.symbol = 'diamond'
    wb.save('gbenergy.xlsx')

    chart.ser[0].marker.size = 10
    chart.ser[1].marker.size = 8
    # chart.ser[2].marker.symbol = 'diamond'
    wb.save('gbenergy.xlsx')

    # マーカーの色を変更する
    chart.ser[0].marker.graphicalProperties.line.solidFill = "FF0000"
    chart.ser[1].marker.graphicalProperties.line.solidFill = "00FF00"
    # chart.ser[2].marker.graphicalProperties.line.solidFill = "0000FF"
    wb.save('gbenergy.xlsx')

    # グラフの種類を変更する
    # chart.grouping = 'stacked'  # 積み上げ折れ線グラフ
    # chart.grouping = 'percentStacked'  # 100％積み上げ折れ線グラフ
    chart.grouping = 'standard'  # 折れ線グラフ
    wb.save('gbenergy.xlsx')

In [41]:
def get_energy(filename):
    n_layer,dx,energy = np.loadtxt(filename,unpack=True)
    min_energy = energy.min()
    return min_energy

In [42]:
def get_cnid_energy(filename):
    a,b,dx,dy,dz,energy,deltax = np.loadtxt(filename,unpack=True)
    min_energy = energy.min()
    return min_energy

In [43]:
def order_indice(test):
    test = np.array(test)
    test = np.sort(test)
    x, y, z = test
    if x!=y:
        test = np.array([z,y,x])
    else:
        pass
    return test

In [44]:
def get_100_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        gbe = get_energy(f"{filename}/results")
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    result_sorted = np.vstack([np.array([0,0]),result_sorted,np.array([90,0])])
    return result_sorted

In [45]:
def get_100_cnid_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        gbe = get_cnid_energy(f"{filename}/results")
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    result_sorted = np.vstack([np.array([0,0]),result_sorted,np.array([90,0])])
    return result_sorted

In [46]:
def get_110_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle)
        gbe = get_energy(f"{filename}/results")   
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    result_sorted = np.vstack([np.array([0,0]),result_sorted,np.array([180,0])])
    return result_sorted

In [47]:
def get_110_cnid_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle)
        gbe = get_cnid_energy(f"{filename}/results")
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    result_sorted = np.vstack([np.array([0,0]),result_sorted,np.array([180,0])])
    return result_sorted

In [48]:
def get_111_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        gbe = get_energy(f"{filename}/results") 
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    return result_sorted

In [49]:
def get_111_cnid_gbenergy(dirname):
    misangle_list = []
    gbe_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        gbe = get_cnid_energy(f"{filename}/results")
        mis_angle = np.round(mis_angle,5)
        gbe = np.round(gbe,5)
        misangle_list.append(mis_angle)
        gbe_list.append(gbe)
    result = np.array([misangle_list,gbe_list]).T
    col_num = 0
    result_sorted = result[np.argsort(result[:, col_num])]
    return result_sorted

In [50]:
def get_100_gbindice(dirname):
    misangle_list = []
    gbindice_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = np.round(mis_angle,5)
        misangle_list.append(mis_angle)
        gbindice_list.append(f"{int(gbname[0])} {int(gbname[1])} {int(gbname[2])}")
        
    misangle_list = np.array(misangle_list)
    gbindice_list = np.array(gbindice_list)
    gbindice_list_sorted = gbindice_list[np.argsort(misangle_list)]
    gbindice_list_sorted = np.hstack([np.array(["1 0 0"]),gbindice_list_sorted,np.array(["1 1 0"])])
    return gbindice_list_sorted

In [51]:
def get_110_gbindice(dirname):
    misangle_list = []
    gbindice_list = []
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle) 
        mis_angle = np.round(mis_angle,5)
        misangle_list.append(mis_angle)
        gbindice_list.append(f"{int(gbname[0])} {int(gbname[1])} {int(gbname[2])}")
        
    misangle_list = np.array(misangle_list)
    gbindice_list = np.array(gbindice_list)
    gbindice_list_sorted = gbindice_list[np.argsort(misangle_list)]
    gbindice_list_sorted = np.hstack([np.array(["1 1 0"]),gbindice_list_sorted,np.array(["1 0 0"])])
        
    return gbindice_list_sorted

In [52]:
def get_111_gbindice(dirname):
    misangle_list = []
    gbindice_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))
    for filename in glob.glob(f"{dirname}/*gb"):
        filename_dash = filename.split("/")[-1]
        gbname = filename_dash.split("_")
        gbname = gbname[:-1]
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        mis_angle = np.round(mis_angle,5)
        misangle_list.append(mis_angle)
        gbindice_list.append(f"{int(gbname[0])} {int(gbname[1])} {int(gbname[2])}")
        
    misangle_list = np.array(misangle_list)
    gbindice_list = np.array(gbindice_list)
    gbindice_list_sorted = gbindice_list[np.argsort(misangle_list)]
    gbindice_list_sorted = np.hstack([np.array(["1 1 0"]),gbindice_list_sorted,np.array(["2 1 1"])])
    
    return gbindice_list_sorted

In [53]:
def make_results_excel(results,results2):
    wb = Workbook()

    for i,dirname in enumerate(glob.glob(f"{results}/*")):
        print(dirname)
        name = dirname.split("/")[-1]
        name_dash = name.split("_")[0]
        axis = "".join(list(name_dash)[-3:])
        print(name)
        print(axis)
        if i == 0:
            ws = wb.active
            ws.title = f"{name}"

        else:
            ws = wb.create_sheet(title = f"{name}")

        if axis=="100":
            result_sorted = get_100_gbenergy(dirname)
            result_sorted_cnid = get_100_cnid_gbenergy(f"{results2}/{name}")
            result_tot = np.hstack([result_sorted,result_sorted_cnid[:,1].reshape(-1,1),]).tolist()
            gbindice_list = get_100_gbindice(dirname)
            result_tot_gbindice = [result+[gbindice] for result,gbindice in zip(result_tot,gbindice_list)]
    #         print(result_tot_gbindice)
            make_excel(wb,ws,name_dash,result_tot_gbindice)
    #         gbname = np.repeat(f"{name}",len(result_sorted))
    #         result_sorted = np.hstack((result_sorted,gbname.reshape(-1,1)))
        elif axis=="110":
            result_sorted = get_110_gbenergy(dirname)
            result_sorted_cnid = get_110_cnid_gbenergy(f"{results2}/{name}")
            result_tot = np.hstack([result_sorted,result_sorted_cnid[:,1].reshape(-1,1),]).tolist()
            gbindice_list = get_110_gbindice(dirname)
            result_tot_gbindice = [result+[gbindice] for result,gbindice in zip(result_tot,gbindice_list)]
            make_excel(wb,ws,name_dash,result_tot_gbindice)

    #         gbname = np.repeat(f"{name}",len(result_sorted))
    #         result_sorted = np.hstack((result_sorted,gbname.reshape(-1,1)))
        else:
            result_sorted = get_111_gbenergy(dirname)
            result_sorted_cnid = get_111_cnid_gbenergy(f"{results2}/{name}")
            result_tot = np.hstack([result_sorted,result_sorted_cnid[:,1].reshape(-1,1),]).tolist()
            gbindice_list = get_111_gbindice(dirname)
            result_tot_gbindice = [result+[gbindice] for result,gbindice in zip(result_tot,gbindice_list)]
            make_excel(wb,ws,name_dash,result_tot_gbindice)

    #         gbname = np.repeat(f"{name}",len(result_sorted))
    #         result_sorted = np.hstack((result_sorted,gbname.reshape(-1,1)))

    #     if first_loop==True:
    #         result_tot = result_sorted
    #         first_loop=False
    #     else:
    #         result_tot = np.vstack((result_tot,result_sorted))

In [54]:
make_results_excel("result","result2")

result/Cu100
Cu100
100
result/Cu110
Cu110
110
result/Cu111
Cu111
111
result/Fe100
Fe100
100
result/Fe110
Fe110
110
result/Fe111
Fe111
111
